# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [40]:
import redis

In [41]:
r = redis.Redis(host='localhost', port=6379)

In [6]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [7]:
r.set('greetings', 'hello world')
value = r.get('greetings')
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [8]:
r.set('connections', 0)
value = r.incr('connections')
print(value)

# value2 = r.incr('greetings')
# print(value2)
# # incrémenter une clé dont la valeur est un string renvoie une erreur

1


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [9]:
r.rpush('colors', *['red', 'black', 'blue'])

3

**Q** : Display the second element of the list with `LINDEX`

In [10]:
r.lindex('colors', 1)

b'black'

**Q** : Display the full list with `LRANGE`

In [11]:
r.lrange('colors', 0, -1)

[b'red', b'black', b'blue']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [12]:
r.sadd('nosql', *['redis', 'mongodb', 'orientdb'])

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [13]:
r.sismember('nosql', 'mysql')

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [14]:
# Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`.
# Try to add `redis` again and see what happens.

r.sadd('nosql', 'hbase')
r.smembers('nosql')

r.sadd('nosql', 'redis')
r.smembers('nosql')

{b'hbase', b'mongodb', b'orientdb', b'redis'}

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [15]:
data = {
    "Agen": 10,
    "Bordeaux": 33,
    "Brive": 32,
    "Castres": 29,
    "Clermont": 38,
    "Grenoble": 24,
    "La Rochelle": 26,
    "Montpellier": 32,
    "Oyonnax": 14,
    "Pau": 20,
    "Racing": 40,
    "Stade Français": 22,
    "Toulon": 36,
    "Toulouse": 36
}

r.zadd('top14', data)

r.zrange('top14', 0, -1, withscores=True)


[(b'Agen', 10.0),
 (b'Oyonnax', 14.0),
 (b'Pau', 20.0),
 (b'Stade Fran\xc3\xa7ais', 22.0),
 (b'Grenoble', 24.0),
 (b'La Rochelle', 26.0),
 (b'Castres', 29.0),
 (b'Brive', 32.0),
 (b'Montpellier', 32.0),
 (b'Bordeaux', 33.0),
 (b'Toulon', 36.0),
 (b'Toulouse', 36.0),
 (b'Clermont', 38.0),
 (b'Racing', 40.0)]

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [16]:
print(r.zscore('top14', 'Toulon'))
print(r.zrank('top14', 'Toulon'))

36.0
10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [17]:
r.zrevrank('top14', 'Toulon')

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [18]:
print(r.zrange('top14', -3, -1, withscores=True))

print(r.zrangebyscore('top14', 35, 1000, withscores=True))

[(b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]
[(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [19]:
r.hmset("user:1", {
    "id": 1,
    "name": "Jean",
    "age": 22
    })

r.hgetall("user:1")

C:\Users\maxen\AppData\Local\Temp\ipykernel_20008\932837088.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset("user:1", {


{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [20]:
# Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

r.hset("user:1", "city", "Lyon")
r.hset("user:1", "name", "Paul")

r.hgetall("user:1")

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [42]:
r.hset("http://my-api.fr/user/10", "PUT", '{"id": 10, "name": "jean"}')

1

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [43]:
r.hset("http://my-api.fr/user?size=2", "GET", '[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]')

1

**Q** : Create a set of cache request keys.

In [44]:
r.sadd("cache_requests", *["http://my-api.fr/user/10", "http://my-api.fr/user?size=2"])

2

**Q** : Check if the GET request on http: //http://my-api.fr/user

In [46]:
r.sismember("cache_requests", "http://my-api.fr/user/10")

# Il n'y a pas de request sur cet url

1

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [51]:
r.hdel("http://my-api.fr/user/10", "PUT")

# Il n'y a plus que la request GET sur cet url

1

In [52]:
r.hgetall("http://my-api.fr/user/10")

{}

In [53]:
r.hgetall("http://my-api.fr/user?size=2")

{b'GET': b'[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]'}

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [39]:
!redis-cli flushall

'redis-cli' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
